In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random





In [3]:
IMAGE_SIZE = [224, 224]
random.seed(42)

In [4]:
# initializing VGG with its own weights,
# include_top false means it removes top dense layers which are optimized for VGG images 
# we need to add new layers and train with our data.
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) # here [3] is 3 channels Red blue Green


ImportError: `load_weights` requires h5py when loading weights from HDF5.

In [10]:
for layer in vgg.layers:
  layer.trainable = False
#Here we Froze the weights of VGG- Meaning we are using existing features from VGG adn mixing with our dense layers

NameError: name 'vgg' is not defined

In [11]:
# Model without any dense layers, features only from VGNET and direct predictions.
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(5, activation='softmax')(x) # here we have 5 output classes so i gave 5 in output layer


NameError: name 'vgg' is not defined

In [12]:
# Creating VGNET model
model = Model(inputs=vgg.input, outputs=prediction)

NameError: name 'vgg' is not defined

In [13]:
model.summary()


NameError: name 'model' is not defined

In [14]:

model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)


NameError: name 'model' is not defined

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [62]:
#Input needs to be (224,224)
# we are addign data augmentation to zoom in out , and flipping horizontally
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)# add augments with horizontal flip

test_datagen = ImageDataGenerator(rescale = 1./255)#divide by 255 to normalize/scale

training_set = train_datagen.flow_from_directory('SecondDataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('SecondDataset/Test',
                                           target_size = (224, 224),
                                             batch_size = 32,
                                             class_mode = 'categorical')

Found 214 images belonging to 5 classes.
Found 81 images belonging to 5 classes.


In [63]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
7/7 [==============================] - 8s 982ms/step - loss: 4.6014 - accuracy: 0.2850 - val_loss: 2.4125 - val_accuracy: 0.4198
Epoch 2/5
7/7 [==============================] - 7s 934ms/step - loss: 2.3912 - accuracy: 0.4065 - val_loss: 0.9441 - val_accuracy: 0.6049
Epoch 3/5
7/7 [==============================] - 7s 942ms/step - loss: 1.7323 - accuracy: 0.4533 - val_loss: 1.0046 - val_accuracy: 0.5556
Epoch 4/5
7/7 [==============================] - 7s 909ms/step - loss: 1.2578 - accuracy: 0.5654 - val_loss: 1.8019 - val_accuracy: 0.5309
Epoch 5/5
7/7 [==============================] - 7s 957ms/step - loss: 1.1292 - accuracy: 0.6822 - val_loss: 0.8106 - val_accuracy: 0.5926


In [64]:
#This did not work very will with VG net architecture.
#Transfer learning may not always work. we need to Tweak some parameters and try it agian,
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 2s 542ms/step - loss: 0.8106 - accuracy: 0.5926
Test Loss: 0.810613214969635
Test accuracy: 0.5925925970077515


In [65]:
# i am adding more train layers to customize the model more.



In [66]:
#Second Try

In [67]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [68]:
for layer in vgg.layers:
  layer.trainable = False

In [69]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
x=Dense(100, activation='relu')(x)
Dropout(0.5)(x)
x=Dense(100, activation='relu')(x)
prediction = Dense(5, activation='softmax')(x)


In [70]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [71]:
model.summary()


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [72]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


In [73]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=16,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),callbacks=callback
)


Epoch 1/16
7/7 [==============================] - 7s 977ms/step - loss: 6.1181 - accuracy: 0.2243 - val_loss: 1.8227 - val_accuracy: 0.2099
Epoch 2/16
7/7 [==============================] - 7s 885ms/step - loss: 1.8560 - accuracy: 0.2196 - val_loss: 1.4662 - val_accuracy: 0.4074
Epoch 3/16
7/7 [==============================] - 6s 905ms/step - loss: 1.4246 - accuracy: 0.4019 - val_loss: 1.0858 - val_accuracy: 0.4815
Epoch 4/16
7/7 [==============================] - 6s 858ms/step - loss: 1.5654 - accuracy: 0.3598 - val_loss: 1.0044 - val_accuracy: 0.4444
Epoch 5/16
7/7 [==============================] - 7s 902ms/step - loss: 1.0257 - accuracy: 0.5514 - val_loss: 1.0359 - val_accuracy: 0.6296
Epoch 6/16
7/7 [==============================] - 7s 890ms/step - loss: 1.5406 - accuracy: 0.4299 - val_loss: 0.9891 - val_accuracy: 0.5309
Epoch 7/16
7/7 [==============================] - 6s 890ms/step - loss: 1.0405 - accuracy: 0.5280 - val_loss: 0.8802 - val_accuracy: 0.5679
Epoch 8/16
7/7 [====

In [74]:
#adding more layers which are trainablle did not help our model.We need to try a different approach.

In [75]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 2s 453ms/step - loss: 0.4251 - accuracy: 0.8765
Test Loss: 0.4251287877559662
Test accuracy: 0.8765432238578796


In [76]:
#Thrid Model - Now i will only use one dense layer and one for output.


In [77]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False

In [78]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
x=Dense(50, activation='relu')(x)
prediction = Dense(5, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [82]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


In [83]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=16,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),callbacks=callback
)


Epoch 1/16
7/7 [==============================] - 7s 945ms/step - loss: 1.6366 - accuracy: 0.3178 - val_loss: 1.4621 - val_accuracy: 0.3457
Epoch 2/16
7/7 [==============================] - 6s 855ms/step - loss: 1.4594 - accuracy: 0.3738 - val_loss: 1.4445 - val_accuracy: 0.3210
Epoch 3/16
7/7 [==============================] - 6s 847ms/step - loss: 1.5083 - accuracy: 0.3411 - val_loss: 1.4679 - val_accuracy: 0.3210
Epoch 4/16
7/7 [==============================] - 6s 867ms/step - loss: 1.4596 - accuracy: 0.3692 - val_loss: 1.4136 - val_accuracy: 0.3580
Epoch 5/16
7/7 [==============================] - 6s 872ms/step - loss: 1.4367 - accuracy: 0.3598 - val_loss: 1.3927 - val_accuracy: 0.3580
Epoch 6/16
7/7 [==============================] - 6s 872ms/step - loss: 1.4389 - accuracy: 0.3598 - val_loss: 1.3856 - val_accuracy: 0.3457
Epoch 7/16
7/7 [==============================] - 7s 902ms/step - loss: 1.4218 - accuracy: 0.3785 - val_loss: 1.3906 - val_accuracy: 0.3580
Epoch 8/16
7/7 [====

In [81]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 2s 468ms/step - loss: 1.2659 - accuracy: 0.3086
Test Loss: 1.2658671140670776
Test accuracy: 0.3086419701576233


In [ ]:
#This model Fitted Best compared to the model whit direct predictions of our data with VGG 
#it worked better than the one with 2 dense layers.

In [31]:
# Its good to use simple models than complex models with more features.

In [137]:
# Here we explored the Power of transfer learning, by using existing features from VGG
# and adding only one dense layer other than output we saved a lot of compute power and time and
#achieved great accuracy.